In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pickle

import re
import nltk
import string
from nltk.corpus import stopwords


from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.preprocessing import MultiLabelBinarizer, LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, hamming_loss

from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score


from sklearn.multiclass import OneVsRestClassifier
from sklearn import metrics


In [2]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rases\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\rases\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\rases\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [3]:
flightData = pd.read_csv('UCRIVERSIDE_20220617 (1).xlsx - EVENTS.csv')


miniDf = flightData[['NARRATIVE REDACTED', 'PHASE 1','PHASE 2', 'PHASE 3']]

miniDf = miniDf.replace(np.nan, '', regex=True)
miniDf['PHASE 1'] = 'Phase 1: ' + miniDf['PHASE 1']
miniDf['PHASE 2'] = 'Phase 2: ' + miniDf['PHASE 2']
miniDf['PHASE 3'] = 'Phase 3: ' + miniDf['PHASE 3']

miniDf['PHASES'] = miniDf[['PHASE 1','PHASE 2', 'PHASE 3']].values.tolist()
# miniDf

miniDf['PHASES']

0          [Phase 1: LANDING, Phase 2: UNKNOWN, Phase 3: ]
1        [Phase 1: UNKNOWN, AIRBORNE, Phase 2: UNKNOWN,...
2        [Phase 1: UNKNOWN, AIRBORNE, Phase 2: , Phase ...
3                 [Phase 1: LANDING, Phase 2: , Phase 3: ]
4        [Phase 1: UNKNOWN, AIRBORNE, Phase 2: UNKNOWN,...
                               ...                        
21243    [Phase 1: VEHICLE / PEDESTRIAN MOVEMENT, Phase...
21244                [Phase 1: TAXI, Phase 2: , Phase 3: ]
21245                [Phase 1: TAXI, Phase 2: , Phase 3: ]
21246                [Phase 1: TAXI, Phase 2: , Phase 3: ]
21247       [Phase 1: TAXI, Phase 2: DEPARTURE, Phase 3: ]
Name: PHASES, Length: 21248, dtype: object

Labeling

In [4]:
miniDf

,NARRATIVE REDACTED,PHASE 1,PHASE 2,PHASE 3,PHASES
0,"SIGNIFICANT OCCURRENCE REPORTED WHEN <N_1>, WM...",Phase 1: LANDING,Phase 2: UNKNOWN,Phase 3:,"[Phase 1: LANDING, Phase 2: UNKNOWN, Phase 3: ]"
1,"NMAC REPORTED WHEN <C_a>, B748, AND <C_b>, B74...","Phase 1: UNKNOWN, AIRBORNE","Phase 2: UNKNOWN, AIRBORNE",Phase 3:,"[Phase 1: UNKNOWN, AIRBORNE, Phase 2: UNKNOWN,..."
2,"<C_a>, PIPER PA31, DESCENDED 700 FEET BELOW AS...","Phase 1: UNKNOWN, AIRBORNE",Phase 2:,Phase 3:,"[Phase 1: UNKNOWN, AIRBORNE, Phase 2: , Phase ..."
3,"<N_1>, PIPER PA46, CLEARED TO LAND RUNWAY 13L...",Phase 1: LANDING,Phase 2:,Phase 3:,"[Phase 1: LANDING, Phase 2: , Phase 3: ]"
4,"<CS_fr>, GULFSTREAM GLF5, CLIMBED 500 FEET AB...","Phase 1: UNKNOWN, AIRBORNE","Phase 2: UNKNOWN, AIRBORNE",Phase 3:,"[Phase 1: UNKNOWN, AIRBORNE, Phase 2: UNKNOWN,..."
...,...,...,...,...,...
21243,PEDESTRIAN ENTERED TWY Z AND Y WITHOUT AUTHORI...,Phase 1: VEHICLE / PEDESTRIAN MOVEMENT,Phase 2:,Phase 3:,"[Phase 1: VEHICLE / PEDESTRIAN MOVEMENT, Phase..."
21244,"<N_1>, CESSNA C172, CROSSED RUNWAY 12L WITHOUT...",Phase 1: TAXI,Phase 2:,Phase 3:,"[Phase 1: TAXI, Phase 2: , Phase 3: ]"
21245,"<N_1>, CESSNA P210, CROSSED RUNWAY 22R HOLD LI...",Phase 1: TAXI,Phase 2:,Phase 3:,"[Phase 1: TAXI, Phase 2: , Phase 3: ]"
21246,"<CS_c>, A21N, ENTERED TWY B, VEHICLE ACCESS RO...",Phase 1: TAXI,Phase 2:,Phase 3:,"[Phase 1: TAXI, Phase 2: , Phase 3: ]"


In [5]:
multilabel = MultiLabelBinarizer()

In [6]:
multilabel.fit(miniDf['PHASES'])

# transform target variable
y = multilabel.transform(miniDf['PHASES'])

In [7]:
multilabel.classes_

array(['Phase 1: ', 'Phase 1: ABORTED TAKE OFF',
       'Phase 1: ABORTED TAKEOFF', 'Phase 1: AERIAL APPLICATION',
       'Phase 1: AERIAL DEMONSTRATION', 'Phase 1: APPROACH',
       'Phase 1: ARRIVAL', 'Phase 1: DEPARTURE', 'Phase 1: EN ROUTE',
       'Phase 1: EN ROUTE OCEANIC', 'Phase 1: LANDING', 'Phase 1: LUAW',
       'Phase 1: MAINTENANCE TAXI', 'Phase 1: MANEUVER',
       'Phase 1: MISSED APPROACH', 'Phase 1: NEC',
       'Phase 1: PARACHUTE OPERATIONS', 'Phase 1: PUSHBACK',
       'Phase 1: STANDING', 'Phase 1: TAKE OFF', 'Phase 1: TAXI',
       'Phase 1: TOUCH AND GO', 'Phase 1: TOW', 'Phase 1: UNKNOWN',
       'Phase 1: UNKNOWN, AIRBORNE', 'Phase 1: UNKNOWN, SURFACE',
       'Phase 1: VEHICLE / PEDESTRIAN MOVEMENT', 'Phase 2: ',
       'Phase 2: APPROACH', 'Phase 2: ARRIVAL', 'Phase 2: DEPARTURE',
       'Phase 2: EN ROUTE', 'Phase 2: LANDING', 'Phase 2: LUAW',
       'Phase 2: MISSED APPROACH', 'Phase 2: PARACHUTE OPERATIONS',
       'Phase 2: STANDING', 'Phase 2: TAKE OFF'

In [8]:
pd.DataFrame(y, columns=multilabel.classes_)

,Phase 1:,Phase 1: ABORTED TAKE OFF,Phase 1: ABORTED TAKEOFF,Phase 1: AERIAL APPLICATION,Phase 1: AERIAL DEMONSTRATION,Phase 1: APPROACH,Phase 1: ARRIVAL,Phase 1: DEPARTURE,Phase 1: EN ROUTE,Phase 1: EN ROUTE OCEANIC,...,Phase 2: UNKNOWN,"Phase 2: UNKNOWN, AIRBORNE","Phase 2: UNKNOWN, SURFACE",Phase 2: VEHICLE / PEDESTRIAN MOVEMENT,Phase 3:,Phase 3: APPROACH,Phase 3: DEPARTURE,Phase 3: LANDING,Phase 3: MISSED APPROACH,Phase 3: TAKE OFF
0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21243,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
21244,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
21245,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
21246,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


Preprocessing

In [9]:
# Stopword removal function
stop_words = stopwords.words('english')

def remove_stop_words(narrative):
  return ' '.join([word for word in narrative.split() if word not in stop_words and len(word) > 1])

In [10]:
narratives = miniDf['NARRATIVE REDACTED']

# Remove all tags from narratives
narratives = [re.sub(r'<(.+?)>','',narrative) for narrative in narratives]

# Stripping all punctuation from narratives
narratives = [narrative.translate(str.maketrans('','',string.punctuation)) for narrative in narratives]

# Remove all stopwords from narratives
narratives = list(map(remove_stop_words,narratives))
narratives = np.array(narratives)
miniDf['NARRATIVE REDACTED'] = narratives

In [11]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=10000)

In [12]:
!pip install iterative-stratification

You should consider upgrading via the 'c:\users\rases\anaconda3\python.exe -m pip install --upgrade pip' command.


In [13]:
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit

msss = MultilabelStratifiedShuffleSplit(n_splits=5, test_size=0.35, random_state=0)

X = miniDf['NARRATIVE REDACTED']

for train_index, test_index in msss.split(X, y):
   print("TRAIN:", train_index, "TEST:", test_index)
   X_train, X_test = X[train_index], X[test_index]
   y_train, y_test = y[train_index], y[test_index]

# xtrain, ytrain, xval, yval = iterative_train_test_split(miniDf['NARRATIVE REDACTED'], y, test_size = 0.35)

TRAIN: [    1     2     5 ... 21245 21246 21247] TEST: [    0     3     4 ... 21239 21240 21242]
TRAIN: [    0     1     2 ... 21244 21245 21247] TEST: [    4     5    11 ... 21240 21242 21246]
TRAIN: [    0     1     2 ... 21241 21245 21246] TEST: [    6     8    10 ... 21243 21244 21247]
TRAIN: [    0     1     4 ... 21244 21245 21247] TEST: [    2     3    10 ... 21238 21240 21246]
TRAIN: [    0     1     2 ... 21245 21246 21247] TEST: [    5     7     8 ... 21238 21239 21243]


In [14]:
xtrain_tfidf = tfidf_vectorizer.fit_transform(X_train) 
xval_tfidf = tfidf_vectorizer.transform(X_test)

In [15]:
SVC = LinearSVC()
clf = OneVsRestClassifier(SVC)

# fit model on train data 
clf.fit(xtrain_tfidf, y_train)

# make predictions for validation set 
y_pred = clf.predict(xval_tfidf)

In [16]:
f1_score(y_test, y_pred, average="micro")

0.9425896965163981

In [17]:
#y_pred = clf.predict(X_test)
print(f"Test Set Accuracy : {accuracy_score(y_test, y_pred) * 100} %\n\n")
print(f"Test Set Loss : {hamming_loss(y_test, y_pred) * 100} %\n\n")
print(f"Classification Report : \n\n{classification_report(y_test, y_pred)}")

Test Set Accuracy : 81.38784292630447 %


Test Set Loss : 0.6550771713493606 %


Classification Report : 

              precision    recall  f1-score   support

           0       0.87      0.68      0.76       323
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         1
           4       0.00      0.00      0.00         2
           5       0.77      0.53      0.63       103
           6       0.86      0.80      0.83        30
           7       0.91      0.81      0.86       499
           8       0.90      0.89      0.90       188
           9       0.81      0.81      0.81        47
          10       0.94      0.81      0.87       345
          11       0.00      0.00      0.00         2
          12       0.00      0.00      0.00         7
          13       0.00      0.00      0.00         6
          14       0.71      0.38      0.50        13
          15       1.00     

C:\Users\rases\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\rases\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [18]:
#Inference Function For Phase Labels

In [19]:
def infer_tags(x):
    x = remove_stop_words(x)
    x = re.sub(r'<(.+?)>','',x)
    x_vec = tfidf_vectorizer.transform([x])
    x_pred = clf.predict(x_vec)
    return multilabel.inverse_transform(x_pred)


In [20]:
for i in range(5): 
  k = X_test.sample(1).index[0] 
  print("Narrative: ", miniDf['NARRATIVE REDACTED'][k], "\nPredicted categories: ", infer_tags(X_test[k])), print("Actual categories: ",miniDf['PHASES'][k])
  print()

Narrative:  IFR CRJ2 REPORTED NMAC WITH UNKN POWER GLIDER AT 4000 FEET 15 NW MINNEAPOLIS MN 102045C CLOSEST PROXIMITY 500 FEET VERTICAL UNKN LATERAL 
Predicted categories:  [('Phase 1: UNKNOWN, AIRBORNE', 'Phase 2: UNKNOWN, AIRBORNE', 'Phase 3: ')]
Actual categories:  ['Phase 1: UNKNOWN, AIRBORNE', 'Phase 2: UNKNOWN, AIRBORNE', 'Phase 3: ']

Narrative:  N1089W BEECH BE36 ENTERED DCA SFRA WITHOUT AUTHORIZATION AT UNKN ALTITUDE 12 SW LEESBURG VA 191712E NO CONFLICTS REPORTED EROC 924 MARTIN 
Predicted categories:  [('Phase 1: ', 'Phase 2: ', 'Phase 3: ')]
Actual categories:  ['Phase 1: ', 'Phase 2: ', 'Phase 3: ']

Narrative:  HAWKER BEECHCRAFT BE35 FAILED TO FOLLOW ATC INSTRUCTIONS AT 2600 FEET SSW WALLA WALLA WA 081303P NO CONFLICTS REPORTED 
Predicted categories:  [('Phase 1: UNKNOWN, AIRBORNE', 'Phase 2: ', 'Phase 3: ')]
Actual categories:  ['Phase 1: UNKNOWN, AIRBORNE', 'Phase 2: ', 'Phase 3: ']

Narrative:  SIGNIFICANT OPERATIONAL OCCURRENCE REPORTED WHEN CESSNA C172 AND EVEKTOR EV

In [21]:
#Creating CSV File

In [22]:
nr = miniDf.iloc[X_test.index]['NARRATIVE REDACTED']
ph = miniDf.iloc[X_test.index]['PHASES']

predicted = pd.concat([nr, ph], axis = 1)

predicted['Predicted'] = predicted.apply(lambda x: infer_tags(x['NARRATIVE REDACTED']), axis=1)

In [23]:
#predicted.to_excel('Actual_VS_Predicted_Phases.xlsx')

In [ ]:
#Create Pickle File

In [24]:
import pickle

In [25]:
filename = 'phase_pred_model'
pickle.dump(clf, open(filename, 'wb'))

In [45]:
load_model = pickle.load(open(filename, 'rb'))
mlb = load_model.predict(xval_tfidf[0])
multilabel.inverse_transform(mlb)

[('Phase 1: UNKNOWN, AIRBORNE', 'Phase 2: ', 'Phase 3: ')]